# 用tensorflow实现CNN中MNIST案例

In [1]:
# -*-coding:
####required libaraies
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./MNIST_data",one_hot=True)

########extract data#####
train_X,train_Y,test_X,test_Y=mnist.train.images,mnist.train.labels,mnist.test.images,\
mnist.test.labels

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


### 在TF中二维卷积和池化函数的定义

In [3]:
%matplotlib inline
############C O N V O L U T I O N # L A Y E R ##########
def conv2d(x,w,b,stride=1):
    x=tf.nn.conv2d(x,w,strides=[1,stride,stride,1],padding='SAME')
    x=tf.add(x,b)
    return tf.nn.relu(x)

############P O O L I N G # L A Y E R ##########
def maxpool2d(x,stride=2):
    x=tf.nn.max_pool(x,ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
    return x

In [4]:
#####set values#####
learning_rate=0.01
batch_size=200
num_batches=int(train_X.shape[0]/batch_size)###注意这里一定要加int，否则循环出错
filter_width=5
filter_height=5
input_width=28
input_height=28
dropout=0.75
epoches=20
n_classes=10
in_depth=1
out_depth1=64
out_depth2=128
show_step=1

## CNN架构是3个卷积层(连同池化层)，一个全连接层(FC layer)，后者就是Keras中的Dense Layer

In [5]:
####variables and placeholders######
x_=tf.placeholder(tf.float32,[None,28*28])
y_=tf.placeholder(tf.float32,[None,n_classes])
keep_prob=tf.placeholder(tf.float32)

Weights={
    "wc1":tf.Variable(tf.random_normal([filter_width,filter_height,in_depth,\
                                       out_depth1]),dtype=tf.float32),
    "wc2":tf.Variable(tf.random_normal([filter_width,filter_height,out_depth1,\
                                       out_depth2]),dtype=tf.float32),
    "wd1":tf.Variable(tf.random_normal([int(input_width*input_height/16*out_depth2),\
                                        1024]),dtype=tf.float32),
    "out":tf.Variable(tf.random_normal([1024,n_classes]),dtype=tf.float32)   
}

biases={
    "bc1":tf.Variable(tf.random_normal([out_depth1]),dtype=tf.float32),
    "bc2":tf.Variable(tf.random_normal([out_depth2]),dtype=tf.float32),
    "bd1":tf.Variable(tf.random_normal([1024]),dtype=tf.float32),
    "out":tf.Variable(tf.random_normal([n_classes]),dtype=tf.float32),
    
}

## 定义卷积神经网络架构=Keras里面的model...

In [6]:
def conv_net(x,w,b,dropout):
    x=tf.reshape(x,[-1,28,28,1])
    
    ####layer 1####
    conv1=conv2d(x,w["wc1"],b["bc1"])
    conv1=maxpool2d(conv1)
    
    ####layer 2#####
    conv2=conv2d(conv1,w["wc2"],b["bc2"])
    conv2=maxpool2d(conv2)
    
    #####fully connected layer####
    fc1=tf.matmul(tf.reshape(conv2,[-1,w["wd1"].get_shape().as_list()[0]]),w["wd1"])
    fc1=tf.add(fc1,b["bd1"])
    fc1=tf.nn.relu(fc1)
    
    #####apply dropout####
    fc1=tf.nn.dropout(fc1,dropout)
    
    #####output#######
    out=tf.add(tf.matmul(fc1,w["out"]),b["out"])
    
    return out

In [7]:
######setting ops#####
pred=conv_net(x_,Weights,biases,keep_prob)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y_))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#####evaluate model#####
correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()###尤其注意这句不能放的靠前
start_time = time.time()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

